In [1]:
from sage.all import *
import random
import os

def cM2_index(edges, degrees):
    total = 0
    for (i, j) in edges:
        total += abs(degrees[i]**2 - degrees[j]**2)
    return total

def is_connected(edges, n):
    if len(edges) < n - 1:
        return False
    G = Graph()
    G.add_vertices(range(1, n+1))
    G.add_edges(edges)
    return G.is_connected()

# Generiranje začetne rešitve
def initial_solution(n, k):
    m = n - 1 + k
    V = list(range(1, n+1))
    random.shuffle(V)
    
    edges = []
    for i in range(1, n):
        u, v = V[i-1], V[i]
        edges.append((min(u,v), max(u,v)))
    
    E_all = [(i,j) for i in range(1,n+1) for j in range(i+1,n+1)]
    E_available = [e for e in E_all if e not in edges]
    
    while len(edges) < m and E_available:
        e = random.choice(E_available)
        edges.append(e)
        E_available.remove(e)
    
    return edges

# Generiranje sosednjega grafa
def generate_neighbor(current_edges, n, k, max_attempts=100):
    m = n - 1 + k
    E_all = [(i,j) for i in range(1,n+1) for j in range(i+1,n+1)]
    E_current = set(current_edges)
    E_available = [e for e in E_all if e not in E_current]
    
    for _ in range(max_attempts):
        if not current_edges or not E_available:
            break
        
        e_remove = random.choice(list(current_edges))
        e_add = random.choice(E_available)
        
        new_edges = [e for e in current_edges if e != e_remove]
        new_edges.append(e_add)
        
        if is_connected(new_edges, n):
            return new_edges
    
    return current_edges

def SA_metaheuristic(n, k, max_iter=10000, T_start=100, T_end=0.01, 
                     alpha=0.95, minimize=True, num_runs=5, verbose=True):
    best_obj = float('inf') if minimize else float('-inf')
    best_edges = None
    best_deg = None
    
    for run in range(num_runs):
        if verbose:
            print(f"\nRun {run+1}/{num_runs}")
        
        # Začetna rešitev
        current_edges = initial_solution(n, k)
        G = Graph()
        G.add_vertices(range(1, n+1))
        G.add_edges(current_edges)
        current_deg = {v: G.degree(v) for v in range(1, n+1)}
        current_obj = cM2_index(current_edges, current_deg)
        
        # Najboljša rešitev v tem run-u
        best_obj_run = current_obj
        best_edges_run = list(current_edges)
        best_deg_run = dict(current_deg)
        
        if verbose:
            print(f"  Initial objective: {current_obj}")
        
        T = T_start
        iteration = 0
        
        # Glavna SA zanka
        while T > T_end and iteration < max_iter:
            iteration += 1
            
            # Generiranje soseda
            neighbor_edges = generate_neighbor(current_edges, n, k)
            
            if neighbor_edges == current_edges:
                T *= alpha
                continue
            
            # Izračun objective soseda
            G_neighbor = Graph()
            G_neighbor.add_vertices(range(1, n+1))
            G_neighbor.add_edges(neighbor_edges)
            neighbor_deg = {v: G_neighbor.degree(v) for v in range(1, n+1)}
            neighbor_obj = cM2_index(neighbor_edges, neighbor_deg)
            
            # Delta
            if minimize:
                delta = neighbor_obj - current_obj
            else:
                delta = current_obj - neighbor_obj
            
            # Metropolis kriterij
            if delta < 0 or random.random() < exp(-delta / T):
                current_edges = neighbor_edges
                current_deg = neighbor_deg
                current_obj = neighbor_obj
                
                # Posodobi najboljšo rešitev v run-u
                if (minimize and current_obj < best_obj_run) or \
                   (not minimize and current_obj > best_obj_run):
                    best_obj_run = current_obj
                    best_edges_run = list(current_edges)
                    best_deg_run = dict(current_deg)
            
            T *= alpha
        
        if verbose:
            print(f"  Run objective: {best_obj_run}")
        
        # Posodobi globalno najboljšo rešitev
        if (minimize and best_obj_run < best_obj) or \
           (not minimize and best_obj_run > best_obj):
            best_obj = best_obj_run
            best_edges = list(best_edges_run)
            best_deg = dict(best_deg_run)
    
    if verbose:
        print(f"\nBest objective: {best_obj}")
        print(f"Edges (m={len(best_edges)}): {sorted(best_edges)}")
        print(f"Degrees: {best_deg}")
    
    return dict(objective=best_obj, edges=best_edges, deg=best_deg)

def draw_graph(res, filename=None, layout='spring'):
    if res is None:
        return
    print(res)

    if 'graph' in res:
        G = res['graph']
    else:
        edges = res['edges']
        G = Graph()
        for (i, j) in edges:
            G.add_edge(i, j)
    
    n = G.num_verts()
    k = G.num_edges() - n + 1
    
    fig = G.plot(
        figsize=6,
        layout=layout,
        vertex_color='black',
        edge_color='black',
        vertex_labels=True,
        title=f"n={n}, k={k}, objective={int(round(res['objective']))}"
    )
    
    if filename:
        directory = os.path.dirname(filename)
        if directory and not os.path.exists(directory):
            os.makedirs(directory)
        
        fig.save(filename)
    
    return fig.show() 

In [0]:
k = 3
n_list = [15, 20, 25, 30, 40, 50, 75, 100]

for n in n_list:
    print(f"n={n}, k={k}")
    print("\nMin:")
    res_min = SA_metaheuristic(
        n, k, 
        max_iter=5000, 
        T_start=100, 
        alpha=0.97,
        minimize=True,  
        num_runs=3, 
        verbose=True
    )
    draw_graph(res_min, filename=f"images/metaheuristic/min_n{n}_k{k}.png")
    print("\nMax:")
    res_max = SA_metaheuristic(
        n, k, 
        max_iter=5000, 
        T_start=100, 
        alpha=0.97,
        minimize=False, 
        num_runs=3, 
        verbose=True
    )
    draw_graph(res_max, filename=f"images/metaheuristic/max_n{n}_k{k}.png")
